In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load the data from the UN FAO DAD-IS DB
# Domestic Animal Diversity Information System (DAD-IS)
dad_is_df = pd.read_csv('../data/102821_FAO_DAD-IS_export.csv')
dad_is_df = dad_is_df.replace({'Duck (domestic)': 'Duck', 'Duck(domestic) Muscovy duck': 'Muscovy Duck',
                               'Goose (domestic)': 'Goose', 'Dromedary Bactrian Camel': 'Bactrian Camel',
                               'Vicuña': 'Vicuna', 'Yak (domestic)': 'Yak'})


dad_is_df.columns.tolist()

['Country',
 'ISO3',
 'Specie',
 'Breed/Most common name',
 'Description Of Specific Uses',
 'Lang',
 'Description',
 'Transboundary name',
 'Other name',
 'Uses',
 'Additional information',
 'Additional information comments',
 'Efabis cultural role comment',
 'Efabis cultural value',
 'Adaptability to specific environment',
 'Specific resistance or tolerance',
 'Specific reproductive characteristic',
 'Special characteristic of product',
 'Other special qualities',
 'Reference for special qualities',
 'Efabis genetic features',
 'Efabis environmental role',
 'Efabis adaptability to marginal land',
 'Body conformation',
 'Coat description',
 'Coat quality',
 'Comb type',
 'Skin colour',
 'Shank and foot colour',
 'Plumage colour',
 'Pattern within feather',
 'Avian classification',
 'Color comments',
 'Efabis main colour',
 'Efabis skin colour',
 'Number of horns males',
 'Number of horns females',
 'Horn shape size and comments',
 'Wither height males',
 'Wither height females',
 'Wei

In [3]:
dad_is_df.Specie.unique()

array(['Ass', 'Cattle', 'Chicken', 'Dromedary', 'Goat', 'Horse', 'Sheep',
       'Yak', 'Pig', 'Buffalo', 'Turkey', 'Guanaco', 'Llama', 'Vicuna',
       'Rabbit', 'Deer', 'Duck', 'Goose', 'Guinea fowl', 'Muscovy duck',
       'Peacock', 'Pigeon', 'Alpaca', 'Cassowary', 'Dog',
       'Bactrian Camel', 'Emu', 'Ostrich', 'Bactrian camel', 'Nandu',
       'Partridge', 'Pheasant', 'Quail', 'Muscovy Duck', 'Guinea pig',
       'American Bison', 'Swallow'], dtype=object)

In [4]:
# Drop everything that is missing data in all columns of interest. 
cols_of_interest = ['Weight males', 'Weight females', 'Birth weight males', 'Birth weight females',
                    'Age maturity males', 'Age maturity females', 'Age breeding males', 'Age breeding females',
                    'Age first parturition AVG', 'Age first parturition MIN', 'Age first parturition MAX',
                    'Parturition interval AVG', 'Parturition interval MIN', 'Parturition interval MAX',
                    'Length productive life', 'Daily gain', 'Carcass weight', 'Dressing percentage',
                    'Litter size AVG', 'Litter size MIN', 'Litter size MAX']
# List of units for columns above, gleaned from GUI interface here:
# https://www.fao.org/dad-is/browse-by-country-and-species/en/
units = ['kg']*4 + ['months']*7 + ['day']*3 + ['year', 'g', 'kg', '%'] + ['number']*3
new_colnames = ['{0} ({1})'.format(c, u) for c, u in zip(cols_of_interest, units)]

# For the remaining columns, zero values are non-sensical - should be NaN for log-scale plotting
for c in cols_of_interest:
    mask = dad_is_df[c] == 0
    dad_is_df.loc[mask, c] = np.nan
    
dad_is_df = dad_is_df.dropna(axis=0, how='all', subset=cols_of_interest)
    
# Calculate the number of young per year from the litter size and interval between litters.
# NOTE: not 100% sure this is what the "Parturition interval" means
litter_size = dad_is_df['Litter size AVG']

# Have to convert 0 interval into NaN otherwise we get infinite young per year
litter_interval = dad_is_df['Parturition interval AVG']

young_per_year = litter_size / (litter_interval/365)
dad_is_df['Young per year AVG (number)'] = young_per_year

col_replacement_dict = dict((c, nc) for c, nc in zip(cols_of_interest, new_colnames))
dad_is_df = dad_is_df.rename(columns=col_replacement_dict)

# Save a new cleaner file with units for the relevant columns.
dad_is_df.to_csv('../data/FAO_DAD-IS_cleaned.csv')
dad_is_df.shape

(5544, 112)

In [5]:
# Save number of individual breeds for the relevant columns
counts_by_species = dad_is_df.groupby('Specie')['Breed/Most common name'].count()
counts_by_species.name = 'breed_count'
counts_by_species.to_csv('../data/FAO_DAD-IS_breed_counts_by_species.csv')

In [6]:
# Print the species for which there are > 100 breed representatives
mask = counts_by_species > 100
species_enough_data = counts_by_species[mask].index.tolist()

In [7]:
# Save quantiles for the relevant columns for the species that have enough data.
new_colnames += ['Young per year AVG (number)']
mask = dad_is_df.Specie.isin(species_enough_data)
quantiles_by_species = dad_is_df[mask].groupby('Specie').quantile([0.005, 0.05, 0.25, 0.5, 0.75, 0.95, 0.995])[new_colnames]
quantiles_by_species.index = quantiles_by_species.index.set_names(['Specie', 'quantile'])
quantiles_by_species.to_csv('../data/FAO_DAD-IS_quantiles_by_species.csv')

In [8]:
quantiles_by_species

Weight males (kg)  Weight females (kg)  \
Specie  quantile                                           
Buffalo 0.005               307.780              300.000   
        0.050               350.000              320.100   
        0.250               429.275              382.250   
        0.500               520.500              448.250   
        0.750               600.000              494.500   
...                             ...                  ...   
Sheep   0.250                50.125               40.000   
        0.500                72.000               51.000   
        0.750                95.000               66.000   
        0.950               120.000               82.910   
        0.995               140.000              100.618   

                  Birth weight males (kg)  Birth weight females (kg)  \
Specie  quantile                                                       
Buffalo 0.005                      16.320                     15.305   
        0.050                      18.012                     16.060   
        0.250                      25.000                     24.200   
        0.500                      30.000                     27.950   
        0.750                      33.000                     30.750   
...                                   ...                        ...   
Sheep   0.250                       2.840                      2.525   
        0.500                       3.500                      3.200   
        0.750                       4.100                      3.750   
        0.950                       5.000                      4.622   
        0.995                      46.000                     44.000   

                  Age maturity males (months)  Age maturity females (months)  \
Specie  quantile                                                               
Buffalo 0.005                           18.22                         20.145   
        0.050                           20.00                         21.000   
        0.250                           24.00                         25.250   
        0.500                           30.00                         30.000   
        0.750                           33.00                         33.405   
...                                       ...                            ...   
Sheep   0.250                            7.00                          8.000   
        0.500                           10.00                         10.000   
        0.750                           12.00                         12.000   
        0.950                           18.00                         18.000   
        0.995                           25.58                         18.850   

                  Age breeding males (months)  Age breeding females (months)  \
Specie  quantile                                                               
Buffalo 0.005                           24.00                         26.660   
        0.050                           24.00                         30.000   
        0.250                           33.80                         32.000   
        0.500                           36.00                         36.000   
        0.750                           45.00                         45.175   
...                                       ...                            ...   
Sheep   0.250                           12.00                         12.000   
        0.500                           18.00                         18.000   
        0.750                           36.00                         48.000   
        0.950                           48.00                         60.000   
        0.995                           60.03                         75.000   

                  Age first parturition AVG (months)  \
Specie  quantile                                       
Buffalo 0.005                                  24.00   
        0.050                                  29.